In [1]:
import sys
sys.path.append('../zoho_data_app')

import main_module as m

import pandas as pd
import gspread
import re
import datetime

import dateutil

%load_ext autoreload
%autoreload 2


# file_loc = r'C:\Users\fajar.fatoni\Documents\Python\Data'
file_loc = r'C:\Users\fajar.fatoni\Documents\Python\Data'

gc = gspread.oauth(
    credentials_filename= r'C:\Users\fajar.fatoni\Documents\Python\Data\Google Credentials\karma-oauth.json'
)

SHEET_ID_SOURCE = '1PiRLPCroErrgZ07dlLdZZxUKzFmSZNnFwLy48MDmK2A'


In [2]:
sheetname = 'Data'

# Open the spreadsheet
spreadsheet_source = gc.open_by_key(SHEET_ID_SOURCE)
worksheet = spreadsheet_source.worksheet(sheetname)
rows = worksheet.get_all_records()
result = pd.DataFrame(rows)
result

KeyboardInterrupt: 

In [ ]:
result.columns

Index(['number', 'ContactID', 'FirstName', 'LastName', 'AddressLine1',
       'AddressLine2', 'Town', 'City', 'Country', 'Post Code', 'Email',
       'Phones', 'Owner', 'Primary', 'Family', 'Nationality', 'DoB',
       'WeddingDate', 'Keywords', 'PanCardNo', 'Account Type',
       'Account Status', 'OdysseyNo', 'Servicing Office', 'club',
       'Account Currency', 'Sheet Name'],
      dtype='object')

In [ ]:
result['Main DB'] = ''
result['Sub DB'] = ''

result.loc[result['Sheet Name'] == 'HOK', 'Main DB'] = 'HOK'
result.loc[result['Sheet Name'] == 'KC 2021', ['Main DB','Sub DB']] = ['KC 2021','KCEU']
result.loc[result['Sheet Name'] == 'KCB-Elite-ENG', ['Main DB','Sub DB']] = ['KCB', 'Elite-Eng']
result.loc[result['Sheet Name'] == 'KCB-ID-Excluded KCID', ['Main DB','Sub DB']] = ['KCB', 'ID-Exc']
result.loc[result['Sheet Name'] == 'KCB-NH-ID', ['Main DB','Sub DB']] = ['KCB', 'ID-NH']
result.loc[result['Sheet Name'] == 'KCB-NH-ENG', ['Main DB','Sub DB']] = ['KCB', 'ENG-NH']
result.loc[result['Sheet Name'] == 'KCB-ENG-Excluded KCID', ['Main DB','Sub DB']] = ['KCB', 'ENG-Exc']
result.loc[result['Sheet Name'] == 'KCB-St.Martin', ['Main DB','Sub DB']] = ['KCB', 'ST.Martin-Elite']
result.loc[result['Sheet Name'] == 'KCE', ['Main DB']] = ['KCDEU']
result.loc[result['Sheet Name'] == 'KCI-Elite-Non India', ['Main DB','Sub DB']] = ['KCI', 'Elite-Non India']
result.loc[result['Sheet Name'] == 'KCI-India', ['Main DB','Sub DB']] = ['KCI', 'India']
result.loc[result['Sheet Name'] == 'KCI-NH-India', ['Main DB','Sub DB']] = ['KCI', 'NH-India']
result.loc[result['Sheet Name'] == 'KCI-NH-Non India', ['Main DB','Sub DB']] = ['KCI', 'NH-Non India']
result.loc[result['Sheet Name'] == 'KCI-Non India', ['Main DB','Sub DB']] = ['KCI', 'Non India']
result.loc[result['Sheet Name'] == 'KCID-ID', ['Main DB','Sub DB']] = ['KCID', 'ID']
result.loc[result['Sheet Name'] == 'KCID-ENG', ['Main DB','Sub DB']] = ['KCID', 'ENG']

In [ ]:
current_date = datetime.date.today()

formatted_date = current_date.strftime("%d-%m-%Y")
result['Date DB'] = formatted_date

result['Tags'] = 'Members'

In [ ]:
# if data dated is string
# using error=cource to prevent N/A value errors
result['DoB'] = pd.to_datetime(result['DoB'], format='mixed', errors='coerce')
result['DoB'] = result['DoB'].dt.strftime("%d %b %Y")

result['WeddingDate'] = pd.to_datetime(result['WeddingDate'], format='mixed', errors='coerce')
result['WeddingDate'] = result['WeddingDate'].dt.strftime("%d %b %Y")

In [ ]:
result['DoB']

0        03 Jan 1973
1        23 Aug 1978
2        10 Jan 1971
3        19 Sep 1973
4        03 Nov 1971
            ...     
85441    09 Mar 1969
85442            NaN
85443            NaN
85444            NaN
85445            NaN
Name: DoB, Length: 85446, dtype: object

In [ ]:
# result.columns
# result.columns = ['number', 'ContactID', 'FirstName', 'LastName', 'AddressLine1',
#        'AddressLine2', 'Town', 'City', 'Country', 'Post Code', 'Email',
#        'Phones', 'Owner', 'Primary', 'Family', 'Nationality', 'DoB',
#        'WeddingDate', 'Account Type', 'Account Status', 'Servicing Office', 
#        'Keywords', 'PanCardNo', 'OdysseyNo', 'segment',
#        'Account Currency', 'Main DB', 'Sub DB', 'Date DB', 'Tags']

In [ ]:
result.rename(columns={
    'segment': 'Member Segment',
}, inplace=True)

In [ ]:
result.fillna('', inplace=True)

In [ ]:
result['Email Old'] = result['Email']

m.clean_space(result, 'Email')
m.email_correction(result, 'Email')

result['Email Correction'] = 'No'
result.loc[result['Email Old']!=result['Email'], 'Email Correction'] = 'Yes'

c:\Users\fajar.fatoni\Documents\Python\Task Offline\member_app\../zoho_data_app\main_module.py:127: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)
c:\Users\fajar.fatoni\Documents\Python\Task Offline\member_app\../zoho_data_app\main_module.py:271: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediat

In [ ]:
result = result.replace(r'\n', '', regex=True)

In [ ]:
result[['Email','Main DB']].loc[result['Email'].str.contains(',', na=False)]

,Email,Main DB


In [ ]:
result.columns

Index(['number', 'ContactID', 'FirstName', 'LastName', 'AddressLine1',
       'AddressLine2', 'Town', 'City', 'Country', 'Post Code', 'Email',
       'Phones', 'Owner', 'Primary', 'Family', 'Nationality', 'DoB',
       'WeddingDate', 'Keywords', 'PanCardNo', 'Account Type',
       'Account Status', 'OdysseyNo', 'Servicing Office', 'club',
       'Account Currency', 'Sheet Name', 'Main DB', 'Sub DB', 'Date DB',
       'Tags', 'Email Old', 'Email Correction'],
      dtype='object')

In [ ]:
result = result.reindex(columns=['number', 'ContactID', 'FirstName', 'LastName', 'AddressLine1',
       'AddressLine2', 'Town', 'City', 'Country', 'Post Code', 'Email Old','Email',
       'Email Correction','Phones', 'Owner', 'Primary', 'Family', 'Nationality', 'DoB',
       'WeddingDate', 'Account Type', 'Account Status', 'Servicing Office',
       'Sheet Name', 'Keywords', 'PanCardNo', 'OdysseyNo',
       'Account Currency', 'Main DB', 'Sub DB', 'Date DB', 'Tags'])

In [ ]:
result.rename(columns={
    'Town': 'State',
    'number': 'Number'
    # Add other renames as needed...
}, inplace=True)

In [ ]:
result.to_excel(file_loc+'\compiled-member.xlsx', index=False)
# result.to_excel(file_loc+'\compiled-member-test.xlsx', index=False)

In [ ]:
result = result.reindex(columns=['Number', 'ContactID', 'FirstName', 'LastName', 'State', 'City', 'Country', 'Post Code','Email',
       'Email Correction','Phones', 'Owner', 'Primary', 'Family', 'Nationality', 'DoB',
       'WeddingDate', 'Account Type', 'Account Status', 'Servicing Office', 'Keywords', 'PanCardNo', 'OdysseyNo',
       'Account Currency', 'Main DB', 'Sub DB', 'Date DB', 'Tags'])

In [ ]:
# row_split = 30000
# file_name = 'compiled-member'
# file_type = 'csv'

# m.split_file(result, row_split, file_name, file_type, file_loc)

result.to_csv(file_loc+'\compiled-member.csv', index=False)
# result.to_csv(file_loc+'\compiled-member-test.csv', index=False)

In [ ]:
# result.loc[result['Email'] == 'elliotstyche@hotmail.co.uk'].to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\test-upload.csv', index=False)